<a href="https://colab.research.google.com/github/arsilva02/librarianproject/blob/main/WebScrapingAndFeatureGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/storygraphExport.csv')

In [3]:
#lets look at the contributers section
df[~df['Contributors'].isna()]['Contributors']

,Contributors
12,Neil Smith (Translator)
15,Frances Riddle (Translator)
17,Lucy Scott (Translator)
20,Jeffrey Angles (Translator)
22,"Stephanie Ortega (Translator), Sophie Haydock ..."
...,...
700,"William Hutson (Contributor), Daveed Diggs (Co..."
712,Cassandra Medcalf (Narrator)
713,Gregory Rabassa (Translator)
714,Carol Brown Janeway


In [4]:
#if a "translator" is found in the contributor section, i will mark it as translated
def translation(df):
    df['Translated'] = df['Contributors'].str.contains(r'\(Translator\)', na=False)
    return df

df = translation(df)
df[df['Translated']]

,Title,Authors,Contributors,ISBN/UID,Format,Read Status,Date Added,Last Date Read,Dates Read,Read Count,...,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,Review,Content Warnings,Content Warning Description,Tags,Owned?,Translated
12,Anxious People,Fredrik Backman,Neil Smith (Translator),9781501160837,hardcover,to-read,2025/02/09,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,True
15,Time of the Flies,Claudia Piñeiro,Frances Riddle (Translator),9781913867867,paperback,to-read,2024/10/20,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,True
17,On a Woman's Madness,Astrid H. Roemer,Lucy Scott (Translator),9781949641431,hardcover,to-read,2025/02/26,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,True
20,Killing Kanoko: Selected Poems of Hiromi Ito,Hiromi Itō,Jeffrey Angles (Translator),9780979975547,paperback,to-read,2025/01/08,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"b-phys, translated, challenge, japanese, bought",Yes,True
22,Red House Alley,Else Jerusalem,"Stephanie Ortega (Translator), Sophie Haydock ...",9781915812360,paperback,to-read,2025/01/29,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,Thirst,Marina Yuszczuk,Heather Cleary (Translator),9780593472064,hardcover,to-read,2025/01/10,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,True
687,Strange Weather in Tokyo,Hiromi Kawakami,Allison Markin Powell (Translator),9781846275081,paperback,to-read,2024/04/03,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"translated, japanese",No,True
698,"1Q84, Vol. 1",Haruki Murakami,Jay Rubin (Translator),NaN,paperback,to-read,2024/01/27,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"translated, japanese",No,True
713,Chronicle of a Death Foretold,Gabriel García Márquez,Gregory Rabassa (Translator),9781400034710,paperback,to-read,2024/02/07,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"translated, spanish",No,True


In [5]:
df[df['Title']=='This Is How You Lose the Time War']

,Title,Authors,Contributors,ISBN/UID,Format,Read Status,Date Added,Last Date Read,Dates Read,Read Count,...,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,Review,Content Warnings,Content Warning Description,Tags,Owned?,Translated
567,This Is How You Lose the Time War,"Max Gladstone, Amal El-Mohtar",NaN,9781534430990,paperback,read,2024/04/09,2024/04/12,2024/04/08-2024/04/12,1.0,...,Yes,Yes,Yes,5.0,NaN,NaN,NaN,"non-translated, borrowed, hugo",No,False


In [6]:
test = pd.concat([df[8:10],df[567:568]])

In [41]:
import requests
from bs4 import BeautifulSoup
import math
import re

In [8]:
book_search = test[['Title','Authors','ISBN/UID']]
book_search

,Title,Authors,ISBN/UID
8,The Sun Also Rises,Ernest Hemingway,NaN
9,The End We Start from,Megan Hunter,9780802126894
567,This Is How You Lose the Time War,"Max Gladstone, Amal El-Mohtar",9781534430990


In [46]:
def scraper(df):
  master = 'app.thestorygraph.com'
  search_master = '/browse?search_term='

  for index, row in df.iterrows():
    title = row['Title']
    author = row['Authors'].replace(", ", " ")
    isbn = row['ISBN/UID']

    if pd.isna(isbn):
      search = f"{title.replace(' ', '%20')}%20{author.replace(' ', '%20')}"
    else:
      search = isbn

    search_url = f"https://{master}{search_master}{search}"
    print(search_url)

    response = requests.get(search_url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    soup = BeautifulSoup(response.content, "html.parser")

    href = soup.find("a", href=re.compile(r"/books/"))["href"]

    if href:
      book_url = f"https://{master}{href}"
      print(book_url)
    else:
      print('No book found')




scraper(book_search)

https://app.thestorygraph.com/browse?search_term=The%20Sun%20Also%20Rises%20Ernest%20Hemingway
https://app.thestorygraph.com/books/b3fa4f03-b614-4060-aae6-9fec34cf7b14
https://app.thestorygraph.com/browse?search_term=9780802126894
https://app.thestorygraph.com/books/0488fc0f-b0a6-45c0-81a2-83b3b54f45e3
https://app.thestorygraph.com/browse?search_term=9781534430990
https://app.thestorygraph.com/books/c9b41386-5e9b-4ee3-98b6-2a61f37043f2


In [69]:

response = requests.get('https://app.thestorygraph.com/browse?search_term=The%20Sun%20Also%20Rises%20Ernest%20Hemingway')
response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
soup = BeautifulSoup(response.content, "html.parser")
data = {}

In [70]:
href = soup.find("a", href=re.compile(r"/books/"))["href"]

In [74]:
href

'/books/b3fa4f03-b614-4060-aae6-9fec34cf7b14'

In [75]:
response = requests.get('https://app.thestorygraph.com'+href)
response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
soup = BeautifulSoup(response.content, "html.parser")

In [76]:
text = soup.find(class_='text-xs min-[520px]:text-sm font-light text-darkestGrey dark:text-grey mt-1').text.strip()

try:
        pages_match = re.search(r"(\d+)\s+pages", text)
        year_match = re.search(r"first pub (\d{4})", text)

        pages = int(pages_match.group(1)) if pages_match else float('nan')
        pub_year = int(year_match.group(1)) if year_match else float('nan')

        print(pages, pub_year)

except (AttributeError, ValueError):
        pages, pub_year =  float('nan'), float('nan')
except Exception as e:
        print(f"An unexpected error occurred: {e}")
        pages, pub_year =  float('nan'), float('nan')

189 1926


In [97]:
tags = soup.find(class_="mt-2 my-1 md:w-10/12").text.strip().split('\n')
tags

['fiction', 'classics', 'literary', 'romance', 'reflective', 'slow-paced']

In [48]:
soup

<!DOCTYPE html>

<html class="system">
<head>
<script data-domain="app.thestorygraph.com" defer="" src="https://blue.thestorygraph.com/js/script.js"></script>
<title>The Sun Also Rises by Ernest Hemingway | The StoryGraph</title>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="zEDGAwZ2gN5B_6R5KDM9OiMzPjk_nPykmnQ3lrr9Iahq9loDACwBgzPUW56x3IA63AtjAigOUDgRelrKh7mLwA" name="csrf-token"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no, viewport-fit=cover" name="viewport"/>
<meta content="default" name="apple-mobile-web-app-status-bar-style"/>
<meta content="summary" name="twitter:card"/>
<meta content="@thestorygraph" name="twitter:site">
<meta content="The Sun Also Rises by Ernest Hemingway" property="og:title">
<meta content="/books/b3fa4f03-b614-4060-aae6-9fec34cf7b14" property="og:url"/>
<meta content="The quintessential novel of the Lost Generation, The Sun Also Rises is one of Ernest Hemingway's ..." property="og:description"/>
<meta content=